In [26]:
import time
import numpy as np
import pandas as pd
from typing import Dict, Union
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException
)
from webdriver_manager.chrome import ChromeDriverManager

# from src.common.consts import CommonConsts
# from src.modules.entities.real_estate import RealEstate
# from src.modules.repositories import RealEstateRepo



# Set up Chrome driver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_argument("--log-level=3")  # Suppress logs
# options.add_argument("--disable-logging")  # Disable logging
# options.add_argument("--silent")  # Silent mode
# options.add_argument("headless")
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
driver.set_page_load_timeout(30)
link_all = [
    "https://batdongsan.com.vn/nha-dat-ban",
    "https://batdongsan.com.vn/nha-dat-cho-thue",
]
real_estate_type = [
    "Bán",
    "Cho thuê"
]
driver.get("https://batdongsan.com.vn/nha-dat-ban")

In [27]:
filter_button = driver.find_element(By.CSS_SELECTOR, "div[data-default-value='Loại nhà đất']")
filter_button.click()

In [41]:
# Get all elem that are li tag
elems = driver.find_elements(By.CSS_SELECTOR, ".mCSB_container li[value]")
re_types = []
for elem in elems:
    # if elem.get_attribute("value") != "0":
        print(elem.get_attribute("value"))
        # print(elem.text)
        # re_types.append(elem.text)
        try:
            ul = elem.find_element(By.CSS_SELECTOR, "ul")
            li_elems = ul.find_elements(By.CSS_SELECTOR, "li")
            for li_elem in li_elems:
                print(f"--li: {li_elem.text}")
            print(f"--ul ")
        except NoSuchElementException:
            ul = None
            print(f"--no ul")
        print("-" * 50)
re_types


324
--li: Chung cư mini, căn hộ dịch vụ
--ul 
--------------------------------------------------
650
--no ul
--------------------------------------------------
362
--li: Nhà riêng
--li: Nhà biệt thự, liền kề
--li: Nhà mặt phố
--li: Shophouse, nhà phố thương mại
--ul 
--------------------------------------------------
41
--no ul
--------------------------------------------------
325
--no ul
--------------------------------------------------
163
--no ul
--------------------------------------------------
575
--no ul
--------------------------------------------------
361
--li: 
--li: 
--ul 
--------------------------------------------------
40
--no ul
--------------------------------------------------
283
--no ul
--------------------------------------------------
44
--li: 
--ul 
--------------------------------------------------
562
--no ul
--------------------------------------------------
45
--no ul
--------------------------------------------------
48
--no ul
---------------------------

[]